In [ ]:
from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torch.utils.data import Dataset
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import copy
#data augmentation and Normalization
train_transform = transforms.Compose([
        transforms.Resize((272,272)),
        transforms.RandomRotation(15,),
        transforms.RandomCrop(256),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.507, 0.487, 0.441], std=[0.267, 0.256, 0.276])
    ])

test_transform = transforms.Compose([
        transforms.Resize(256),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.507, 0.487, 0.441], std=[0.267, 0.256, 0.276])
    ])

trainset = datasets.STL10(
    root="data",
    split='train',
    download=True,
    transform=train_transform
)
testset = datasets.STL10(
    root="data",
    split='test',
    download=True,
    transform=test_transform
)
from torch.utils.data import DataLoader

train_dataloader = DataLoader(trainset, batch_size=24, shuffle=True)
test_dataloader = DataLoader(testset, batch_size=24, shuffle=True)
dataloaders={'train': train_dataloader,'test': test_dataloader}
dataset_sizes= {'train':len(trainset),'test':len(testset)}
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_ft = models.wide_resnet101_2()
Num_class=10





Extracting data/stl10_binary.tar.gz to data
Files already downloaded and verified


In [ ]:
def evaluate(model, test_loader,criterion):
    model.eval()
    correct = 0 
    loss = 0
    with torch.no_grad():
        for data, labels in test_loader:
            data = data.cuda()
            output = model(data)
            predicted = torch.max(output,1)[1]
            correct += (predicted == labels.cuda()).sum()
            loss += criterion(output, labels.cuda()).item()

    return (float(correct)/len(testset)) *100, (loss/len(test_loader))

In [ ]:
def train_supervised(model, train_loader, test_loader,criterion, optimizer, num_epochs=25):
    model.train()
    for epoch in range(num_epochs):
        correct = 0
        running_loss = 0
        for batch_idx, (X_batch, y_batch) in enumerate(train_loader):
            X_batch, y_batch = X_batch.cuda(), y_batch.cuda()
            
            output = model(X_batch)
            labeled_loss = criterion(output, y_batch)
                       
            optimizer.zero_grad()
            labeled_loss.backward()
            optimizer.step()
            running_loss += labeled_loss.item()
        if epoch %10 == 0:
            test_acc, test_loss = evaluate(model, test_loader,criterion)
            print('Epoch: {} : Train Loss : {:.5f} | Test Acc : {:.5f} | Test Loss : {:.3f} '.format(epoch, running_loss/(10 * len(trainset)), test_acc, test_loss))
            model.train()
    return model

In [ ]:
model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.001)

# Decay LR by a factor of 0.1 every 7 epochs
#exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

model_ft=train_supervised(model_ft, train_dataloader, test_dataloader,criterion, optimizer_ft,
                       num_epochs=100)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch: 0 : Train Loss : 0.00840 | Test Acc : 28.90000 | Test Loss : 1.789 
Epoch: 10 : Train Loss : 0.00542 | Test Acc : 51.27500 | Test Loss : 1.342 
Epoch: 20 : Train Loss : 0.00436 | Test Acc : 61.62500 | Test Loss : 1.054 
Epoch: 30 : Train Loss : 0.00336 | Test Acc : 68.56250 | Test Loss : 0.890 
Epoch: 40 : Train Loss : 0.00252 | Test Acc : 70.80000 | Test Loss : 0.865 
Epoch: 50 : Train Loss : 0.00176 | Test Acc : 71.73750 | Test Loss : 0.952 
Epoch: 60 : Train Loss : 0.00119 | Test Acc : 74.67500 | Test Loss : 0.843 
Epoch: 70 : Train Loss : 0.00086 | Test Acc : 75.36250 | Test Loss : 0.971 
Epoch: 80 : Train Loss : 0.00054 | Test Acc : 76.92500 | Test Loss : 1.057 
Epoch: 90 : Train Loss : 0.00048 | Test Acc : 76.35000 | Test Loss : 1.072 


In [ ]:
test_acc, test_loss = evaluate(model_ft, test_dataloader,criterion)
print('Test Acc : {:.5f} | Test Loss : {:.3f} '.format(test_acc, test_loss))
torch.save(model_ft.state_dict(), 'supervised.pth')

Test Acc : 76.27500 | Test Loss : 1.079 
